# 데이터 전처리 방안 수정

1) 피처 삭제의 역효과 (`shop_type`, `shop_category`, `shop_type` 제거)
    - 다수의 실험을 통해 `feature_importance`가 낮게 나온 범주형 변수를 삭제하였고 이로 인해 점수가 소폭 하락한 것으로 추정
    - 범주형 변수의 계층 구조 정보를 포함하는 변수를 복원
2) 결측치 처리 방법 수정
    - 이전까지 lag 변수 생성 이후 결측값에 대해 `fillna(0)`로 처리
    - `date_block_num < 3` 인 데이터를 drop 하는 방법 시도(초기시점 데이터의 중요도가 상대적으로 낮을 것으로 예상)

# Module

In [1]:
import pandas as pd
import numpy as np
from itertools import product
import joblib
from sklearn.preprocessing import LabelEncoder

# Data

In [2]:
data_path = "./data/"

sales_train = pd.read_csv(data_path + 'sales_train.csv')
items = pd.read_csv(data_path + 'items.csv')
item_categories = pd.read_csv(data_path + 'item_categories.csv')
shops = pd.read_csv(data_path + 'shops.csv')

test = pd.read_csv(data_path + 'test.csv')
sub = pd.read_csv(data_path + 'sample_submission.csv')

print('Before Filter ShopID:', len(sales_train))
unique_test_shop_id = test['shop_id'].unique()
sales_train = sales_train[sales_train['shop_id'].isin(unique_test_shop_id)]
print('After Filter ShopID :', len(sales_train))

Before Filter ShopID: 2935849
After Filter ShopID : 2413246


In [3]:
def downcast(df, verbose=True):
    start_memory = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col]%1==0).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        elif dtype_name.startswith('float'):
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_memory = df.memory_usage().sum() / 1024**2
    
    if verbose:
        print(f"Memory usage reduced from {start_memory:.2f} MB to {end_memory:.2f} MB")
        
    return df


data_files = [sales_train, items, shops, item_categories, test]
for file in data_files:
    file = downcast(file)

Memory usage reduced from 128.88 MB to 69.04 MB
Memory usage reduced from 0.51 MB to 0.23 MB
Memory usage reduced from 0.00 MB to 0.00 MB
Memory usage reduced from 0.00 MB to 0.00 MB
Memory usage reduced from 4.90 MB to 1.43 MB


In [4]:
train = []

for i in sales_train['date_block_num'].unique():
    all_shop = sales_train.loc[sales_train['date_block_num'] == i, 'shop_id'].unique()
    all_item = sales_train.loc[sales_train['date_block_num'] == i, 'item_id'].unique()
    train.append(np.array(list(product([i], all_shop, all_item))))

# idx features        
idx_features = ['date_block_num', 'shop_id', 'item_id']
train = pd.DataFrame(np.vstack(train), columns=idx_features)

test['date_block_num'] = 34
train.head()

,date_block_num,shop_id,item_id
0,0,59,22154
1,0,59,2552
2,0,59,2554
3,0,59,2555
4,0,59,2564


# Feature Engineering

## Outliers

In [5]:
print("Before : ",len(sales_train))
sales_train = sales_train[(sales_train['item_cnt_day'] > 0)&(sales_train['item_cnt_day'] < 1000)]
print("After item_cnt_day : ",len(sales_train))
sales_train = sales_train[(sales_train['item_price'] > 0)&(sales_train['item_price'] < 50000)]
print("After item_price : ",len(sales_train))

Before :  2413246
After item_cnt_day :  2407079
After item_price :  2407076


## Feature Generation

In [6]:
def add_mean_features(df: pd.DataFrame, groupby_features: list, mean_feature_list: list = None):
    col_name = [col for col in groupby_features if col != 'date_block_num']
    base_name = '_'.join([col.replace('_id', '') for col in col_name])
    feature_name = f'{base_name}_avg_date_sales'
    
    agg_rules = {
        feature_name: ('item_cnt_month', 'mean')
    }
            
    group = df.groupby(groupby_features).agg(**agg_rules).reset_index()
    
    df = df.merge(group, on=groupby_features, how='left')
    df = downcast(df, verbose=False)
    del group
    
    if mean_feature_list is not None:
        mean_feature_list.append(feature_name)
    
    return df


def add_lag_features(df: pd.DataFrame, key_features: list, lag_features_info: dict, lag_period: list):
    """
    lag_features_info: {'feature_name_1': clip_flag_1, 'feature_name_2': clip_flag_2, ...}
    """
    
    group_keys = [key for key in key_features if key != 'date_block_num']
    grouped = df.groupby(group_keys)
    
    all_lag_cols = []
    features_to_clip = []

    for lag_feature, clip_flag in lag_features_info.items():
        for i in lag_period:
            lag_feature_name = f'{lag_feature}_lag_{i}'
            df[lag_feature_name] = grouped[lag_feature].shift(i)
            
            all_lag_cols.append(lag_feature_name)
            if clip_flag:
                features_to_clip.append(lag_feature_name)

    df[all_lag_cols] = df[all_lag_cols]
    
    return df, features_to_clip

### shops

In [7]:
# 데이터 수정
sales_train.loc[sales_train['shop_id']==0, 'shop_id'] = 57
sales_train.loc[sales_train['shop_id']==1, 'shop_id'] = 58
sales_train.loc[sales_train['shop_id']==10, 'shop_id'] = 11
sales_train.loc[sales_train['shop_id']==39, 'shop_id'] = 40

test.loc[test['shop_id']==0, 'shop_id'] = 57
test.loc[test['shop_id']==1, 'shop_id'] = 58
test.loc[test['shop_id']==10, 'shop_id'] = 11
test.loc[test['shop_id']==39, 'shop_id'] = 40

# 도시 구분
shops['city'] = shops['shop_name'].str.split(' ').str[0]
shops.loc[shops['city'] == '!Якутск', 'city'] = 'Якутск'

# 쇼핑몰/센터를 나타내는 러시아어 약어로 상점 구분
mall_keywords = ['ТЦ', 'ТРК', 'ТРЦ', 'MALL', 'Молл']
is_mall = shops['shop_name'].apply(lambda x: any(keyword in x for keyword in mall_keywords))

shops['shop_type'] = np.where(is_mall, 'Mall', 'Standalone')

# 온라인, 오프라인, 이동식 매장 등 특수 카테고리 분류
conditions = [
    shops['shop_name'].str.contains('Интернет-магазин|Выездная Торговля', case=False, regex=True)
]

choices = ['Online']
shops['shop_category'] = np.select(conditions, choices, default='Offline')

encoder = LabelEncoder()
for col in ['city', 'shop_type', 'shop_category']:
    shops[col] = encoder.fit_transform(shops[col])

shops = shops.drop(columns=["shop_name"])

`shop_type`, `shop_category` 변수를 다시 추가

### items

In [8]:
items = items.drop(columns=["item_name"])
items['first_sale_month'] = sales_train.groupby('item_id').agg({'date_block_num': 'min'})['date_block_num']
items = items.fillna(34)

### item_categories

In [9]:
def split_categories(df):
    def split_func(name):
        if '-' in name:
            return name.split('-', 1)
        else:
            return [name, name]
    
    split_result = df['item_category_name'].apply(lambda x: split_func(x))
    df['type'] = split_result.apply(lambda x: x[0].strip())
    df['subtype'] = split_result.apply(lambda x: x[1].strip())
    return df


item_categories = split_categories(item_categories)
etc_types = item_categories['type'].value_counts()[item_categories['type'].value_counts()<5].index.tolist()
item_categories.loc[item_categories['type'].isin(etc_types), 'type'] = 'etc'
item_categories['type'] = LabelEncoder().fit_transform(item_categories['type'])
item_categories['subtype'] = LabelEncoder().fit_transform(item_categories['subtype'])
item_categories = item_categories.drop('item_category_name', axis=1)

### set up matrix

In [10]:
group = sales_train.groupby(idx_features).agg(
    item_cnt_month = ('item_cnt_day', 'sum'),
    transaction_cnt = ('item_cnt_day', 'count'),
    avg_item_price = ('item_price', 'mean'),
).reset_index()

train = train.merge(group, on=idx_features, how='left')
all_data = pd.concat([train, test.drop('ID', axis=1)], ignore_index=True, keys=idx_features)
all_data = all_data.fillna(0)

all_data = all_data.merge(shops, on='shop_id', how='left')
all_data = all_data.merge(items, on='item_id', how='left')
all_data = all_data.merge(item_categories, on='item_category_id', how='left')
all_data.head()

/tmp/ipykernel_37630/1607463284.py:8: FutureWarning: The behavior of pd.concat with len(keys) != len(objs) is deprecated. In a future version this will raise instead of truncating to the smaller of the two sequences
  all_data = pd.concat([train, test.drop('ID', axis=1)], ignore_index=True, keys=idx_features)


,date_block_num,shop_id,item_id,item_cnt_month,transaction_cnt,avg_item_price,city,shop_type,shop_category,item_category_id,first_sale_month,type,subtype
0,0,59,22154,1.0,1.0,999.0,30,0,0,37,0.0,4,1
1,0,59,2552,0.0,0.0,0.0,30,0,0,58,0.0,6,29
2,0,59,2554,0.0,0.0,0.0,30,0,0,58,0.0,6,29
3,0,59,2555,0.0,0.0,0.0,30,0,0,56,0.0,6,5
4,0,59,2564,0.0,0.0,0.0,30,0,0,59,0.0,6,42


In [11]:
del shops, items, item_categories, group

all_data = downcast(all_data)

Memory usage reduced from 714.34 MB to 184.89 MB


In [12]:
all_data.groupby(['shop_category']).agg({'shop_id': 'nunique'})

,shop_id
shop_category,
0,43
1,1


In [13]:
all_data.groupby(['shop_type']).agg({'shop_id': 'nunique'})

,shop_id
shop_type,
0,34
1,10


- `shop_category`는 데이터의 분포가 극도로 불균형하여 이로 인한 정보획득량이 매우 적어 제거

In [14]:
all_data = all_data.drop(columns='shop_category')

### mean features

In [15]:
clip_list = []

all_data = add_mean_features(all_data, ['date_block_num','shop_id'])
all_data = add_mean_features(all_data, ['date_block_num','shop_id','item_category_id'])

all_data = add_mean_features(all_data, ['date_block_num','item_id'])

all_data = add_mean_features(all_data, ['date_block_num', 'item_category_id'])
all_data = add_mean_features(all_data, ['date_block_num', 'item_category_id', 'shop_type'])

In [16]:
mean_feature_list = [col for col in all_data.columns if '_avg_date_sales' in col]
mean_feature_list

['shop_avg_date_sales',
 'shop_item_category_avg_date_sales',
 'item_avg_date_sales',
 'item_category_avg_date_sales',
 'item_category_shop_type_avg_date_sales']

### lag features

In [17]:
%%time
lag_period = [1,2,3]

lag_features_to_process = {
    'item_cnt_month': True,
    'transaction_cnt': False,
    'avg_item_price': False
}

all_data, clip_list = add_lag_features(all_data, 
                                        key_features=idx_features,
                                        lag_features_info=lag_features_to_process,
                                        lag_period=lag_period)

all_data = downcast(all_data)

features_to_drop = []
features_to_drop.extend(['transaction_cnt', 'avg_item_price'])

Memory usage reduced from 949.65 MB to 747.96 MB
CPU times: user 3.66 s, sys: 1.1 s, total: 4.76 s
Wall time: 4.76 s


In [18]:
%%time
# Adding lag features for mean-based aggregations
for mean_feat in mean_feature_list:
    lag_features_to_process[mean_feat] = False
    
all_data, clip_list = add_lag_features(all_data, 
                                        key_features=idx_features,
                                        lag_features_info=lag_features_to_process,
                                        lag_period=lag_period)

all_data = downcast(all_data)

Memory usage reduced from 1453.89 MB to 1252.20 MB
CPU times: user 6.18 s, sys: 2.12 s, total: 8.3 s
Wall time: 8.3 s


In [19]:
# add average of lagged sales
sales_lag_cols = [col for col in all_data.columns if 'item_cnt_month_lag_' in col]
all_data['avg_item_cnt_month_for_lag'] = all_data[sales_lag_cols].mean(axis=1)
all_data[clip_list + ['item_cnt_month', 'avg_item_cnt_month_for_lag']] = all_data[clip_list + ['item_cnt_month', 'avg_item_cnt_month_for_lag']].clip(0, 20)

# add ratio of each lagged sale to average
for col in sales_lag_cols:
    all_data[f'ratio_{col}_to_avg_lag'] = all_data[col] / all_data['avg_item_cnt_month_for_lag']

# add average of lagged prices
price_lag_cols = [col for col in all_data.columns if 'avg_item_price_lag_' in col]
all_data['avg_item_price_for_lag'] = all_data[price_lag_cols].mean(axis=1)

# add ratio of each lagged price to average
for col in price_lag_cols:
    all_data[f'ratio_{col}_to_avg_lag'] = all_data[col] / all_data['avg_item_price_for_lag']

In [20]:
# add average of lagged mean features
for mean_feat in mean_feature_list:
    lag_cols = [col for col in all_data.columns if f'{mean_feat}_lag_' in col]
    feat_name = f'avg_{mean_feat}_lag'
    all_data[feat_name] = all_data[lag_cols].mean(axis=1)

all_data = downcast(all_data)
features_to_drop.extend(mean_feature_list)

Memory usage reduced from 1823.67 MB to 1714.42 MB


### price trend and sales trend

In [21]:
def add_trend_feature(df: pd.DataFrame, base_col: str, lags: list):
    for i in lags:
        col_lag_curr = f'{base_col}_lag_{i}'      
        col_lag_prev = f'{base_col}_lag_{i+1}'    
        col_trend = f'delta_{i}_{base_col}'   
        
        if col_lag_prev not in df.columns or col_lag_curr not in df.columns:
            print(f'[Skip] {col_lag_curr} 또는 {col_lag_prev} 컬럼이 없어 작업을 종료합니다.')
            continue
            
        df[col_trend] = (df[col_lag_curr] - df[col_lag_prev]) / df[col_lag_prev]
        
        df[col_trend] = df[col_trend].replace([np.inf, -np.inf], np.nan)
        df[col_trend] = df[col_trend].fillna(0)
        
    df = downcast(df)

    return df

In [22]:
all_data = add_trend_feature(all_data, base_col='avg_item_price', lags=[1,2])
all_data = add_trend_feature(all_data, base_col='item_cnt_month', lags=[1,2])
all_data = add_trend_feature(all_data, base_col='transaction_cnt', lags=[1,2])

Memory usage reduced from 1848.88 MB to 1781.65 MB
Memory usage reduced from 1848.88 MB to 1848.88 MB
Memory usage reduced from 1916.11 MB to 1916.11 MB


### new item

In [23]:
all_data['new_item'] = all_data['first_sale_month'] == all_data['date_block_num']
all_data['item_age'] = all_data['date_block_num'] - all_data['first_sale_month']
features_to_drop.append('first_sale_month')

### since last sale date

In [24]:
all_data['temp_last_sale'] = np.nan
all_data.loc[all_data['item_cnt_month']>0, 'temp_last_sale'] = all_data['date_block_num']

last_sale_record = all_data.groupby(['item_id', 'shop_id'])['temp_last_sale'].shift(1).ffill()

all_data['since_last_sale'] = (all_data['date_block_num'] - last_sale_record).fillna(-1)
all_data = all_data.drop(columns='temp_last_sale')

### days & month

In [25]:
all_data['month'] = all_data['date_block_num'] % 12
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
all_data['days'] = all_data['month'].map(days)

In [26]:
# seasonal average features
all_data['month_avg_sales'] = np.nan
all_data['month_avg_price'] = np.nan

for month_t in all_data['date_block_num'].unique():
    mask = all_data[all_data['date_block_num'] < month_t]
    if not mask.empty:
        group = mask.groupby(['month', 'item_category_id']).agg(
            month_avg_sales=('item_cnt_month', 'mean'),
            month_avg_price=('avg_item_price', 'mean')
        ).reset_index()
        
        group['key'] = group['month'].astype(str) + '_' + group['item_category_id'].astype(str)
        sales_map_dict = dict(zip(group['key'], group['month_avg_sales']))
        price_map_dict = dict(zip(group['key'], group['month_avg_price']))
        
        current_month = (all_data['date_block_num'] == month_t)
        current_month_keys = all_data.loc[current_month, 'month'].astype(str) + '_' + all_data.loc[current_month, 'item_category_id'].astype(str)
        
        all_data.loc[current_month, 'month_avg_sales'] = current_month_keys.map(sales_map_dict)
        all_data.loc[current_month, 'month_avg_price'] = current_month_keys.map(price_map_dict)

all_data['month_avg_price'] = all_data['month_avg_price'].fillna(all_data['month_avg_price'].mean())
all_data['month_avg_sales'] = all_data['month_avg_sales'].fillna(all_data['month_avg_sales'].mean())

all_data = downcast(all_data)

Memory usage reduced from 2210.25 MB to 2058.98 MB


### removing columns

In [27]:
features_to_drop

['transaction_cnt',
 'avg_item_price',
 'shop_avg_date_sales',
 'shop_item_category_avg_date_sales',
 'item_avg_date_sales',
 'item_category_avg_date_sales',
 'item_category_shop_type_avg_date_sales',
 'first_sale_month']

In [28]:
all_data = all_data.drop(columns=features_to_drop)
all_data.columns

Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month', 'city',
       'shop_type', 'item_category_id', 'type', 'subtype',
       'item_cnt_month_lag_1', 'item_cnt_month_lag_2', 'item_cnt_month_lag_3',
       'transaction_cnt_lag_1', 'transaction_cnt_lag_2',
       'transaction_cnt_lag_3', 'avg_item_price_lag_1', 'avg_item_price_lag_2',
       'avg_item_price_lag_3', 'shop_avg_date_sales_lag_1',
       'shop_avg_date_sales_lag_2', 'shop_avg_date_sales_lag_3',
       'shop_item_category_avg_date_sales_lag_1',
       'shop_item_category_avg_date_sales_lag_2',
       'shop_item_category_avg_date_sales_lag_3', 'item_avg_date_sales_lag_1',
       'item_avg_date_sales_lag_2', 'item_avg_date_sales_lag_3',
       'item_category_avg_date_sales_lag_1',
       'item_category_avg_date_sales_lag_2',
       'item_category_avg_date_sales_lag_3',
       'item_category_shop_type_avg_date_sales_lag_1',
       'item_category_shop_type_avg_date_sales_lag_2',
       'item_category_shop_type_avg_da

In [29]:
all_data.shape

(8812244, 59)

## drop rows with missing values

In [30]:
null_check = all_data.isna().sum()
null_check[null_check > 0]

item_cnt_month_lag_1                               867387
item_cnt_month_lag_2                              1609716
item_cnt_month_lag_3                              2286539
transaction_cnt_lag_1                              867387
transaction_cnt_lag_2                             1609716
transaction_cnt_lag_3                             2286539
avg_item_price_lag_1                               867387
avg_item_price_lag_2                              1609716
avg_item_price_lag_3                              2286539
shop_avg_date_sales_lag_1                          867387
shop_avg_date_sales_lag_2                         1609716
shop_avg_date_sales_lag_3                         2286539
shop_item_category_avg_date_sales_lag_1            867387
shop_item_category_avg_date_sales_lag_2           1609716
shop_item_category_avg_date_sales_lag_3           2286539
item_avg_date_sales_lag_1                          867387
item_avg_date_sales_lag_2                         1609716
item_avg_date_

- `add_lag_features` 함수에서 `.fillna(0)`을 제거하여 초기 3개월 데이터에 대해 결측치를 채우지 않았음
- 생성한 lag feature들에서 시차에 따라 일정하게 결측치가 관찰됨
- 초기 3개월 데이터를 제거하고, 이후 데이터에 대해서 결측치를 0으로 채움

In [31]:
all_data = all_data[all_data['date_block_num']>=3].fillna(0)
all_data.shape

(8026950, 59)

In [32]:
joblib.dump(all_data, data_path + 'all_data2.joblib')

['./data/all_data2.joblib']